In [11]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class TwoPathConv(nn.Module):
    def __init__(self):
        super(TwoPathConv, self).__init__()
        self.upper_layer1 = nn.Sequential(
            nn.Conv2d(4,64,7),
            nn.ReLU(),
            nn.MaxPool2d((4,4),stride = 1)
        )
        self.upper_layer2 = nn.Sequential(
            nn.Conv2d(64,64,3),
            nn.ReLU(),
            nn.MaxPool2d((2,2),stride = 1)
        )
        self.under_layer1 = nn.Sequential(
            nn.Conv2d(4,160,13),
            nn.ReLU()
        )
        self.final_layer = nn.Conv2d(224,5,21)
        
    def forward(self, x):
        upper_x = self.upper_layer2(self.upper_layer1(x))
        under_x = self.under_layer1(x)
        #upper_x = F.max_pool2d(F.relu(self.upper_conv1(x)), (4, 4),stride = 1)
        #upper_x = F.max_pool2d(F.relu(self.upper_conv2(upper_x)), (2, 2), stride = 1)
        #under_x = F.relu(self.under_conv1(x))
        final_x = torch.cat((under_x, upper_x), 1)
        out = self.final_layer(final_x)
        return out
        
net = TwoPathConv()
print(net)

x = Variable(torch.randn(1,4,33,33), requires_grad = True)
y_pred = net.forward(x)
for item in list(net.final_layer.parameters()):
    print(item)
#y_pred = y_pred.data.resize_(1,5)
#y_pred = Variable(y_pred,requires_grad = True)
#print(y_pred.size())
#print(y_pred.view)

TwoPathConv (
  (upper_layer1): Sequential (
    (0): Conv2d(4, 64, kernel_size=(7, 7), stride=(1, 1))
    (1): ReLU ()
    (2): MaxPool2d (size=(4, 4), stride=(1, 1), dilation=(1, 1))
  )
  (upper_layer2): Sequential (
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU ()
    (2): MaxPool2d (size=(2, 2), stride=(1, 1), dilation=(1, 1))
  )
  (under_layer1): Sequential (
    (0): Conv2d(4, 160, kernel_size=(13, 13), stride=(1, 1))
    (1): ReLU ()
  )
  (final_layer): Conv2d(224, 5, kernel_size=(21, 21), stride=(1, 1))
)
Parameter containing:
( 0 , 0 ,.,.) = 
 -2.0225e-03  2.0103e-03 -8.7108e-04  ...   2.5124e-03  1.0436e-03  2.7271e-03
 -3.2380e-04 -2.3371e-03 -2.4914e-03  ...  -9.3090e-04  2.3428e-04  2.9733e-04
 -1.1450e-03 -2.5839e-03  5.4721e-05  ...   1.8655e-03 -2.5009e-03 -3.0827e-03
                 ...                   ⋱                   ...                
  8.5974e-04 -2.9496e-04 -3.7688e-04  ...   1.9733e-03 -2.1250e-03  2.5020e-03
  1.0631e-03  2.7

In [3]:
#get the training set for phase 1
f=open("trainval-balanced.txt", "r")
content=f.readlines()
data=[]
data_train_phase1=[]
i=0
for line in content:
    no_n_line=line[0:len(line)-1]
    item=no_n_line.split(" ")
    if item[0]!="HG/0001":
        data.append([])
        data[i].append(item[0])
        data[i].append(int(item[1]))
        data[i].append(int(item[2]))
        data[i].append(int(item[3]))
        data[i].append(int(item[4]))
        data_train_phase1.append(data[i])
        i += 1
f.close()
from random import shuffle

print ("phase 1 data preparation process completed.")

#get the training set for phase 2 and the validation set
data_val=[]
f_in=open("trainval.txt", "r")
content=f_in.readlines()
data=[]
data_val=[]
data_train_phase2=[]
i=0
for line in content:
    no_n_line=line[0:len(line)-1]
    item=no_n_line.split(" ")
    if item[0]!="HG/0001":
        data.append([])
        data[i].append(item[0])
        data[i].append(int(item[1]))
        data[i].append(int(item[2]))
        data[i].append(int(item[3]))
        data[i].append(int(item[4]))
        if i%30000==0:
            data_val.append(data[i])
        else:
            data_train_phase2.append(data[i])
        i += 1
f_in.close()
print ("phase 2 data preparation process completed.")


import h5py
f = h5py.File('training.h5','r')

phase 1 data preparation process completed.
phase 2 data preparation process completed.


In [4]:
print (len(data))
print (len(data_val))
print (len(data_train_phase1))
print (len(data_train_phase2))

23709634
791
8629298
23708843


In [5]:
print (torch.cuda.is_available())

True


In [6]:
import torch.nn.init as ninit
def init_net(net):
    for param in net.parameters():
        ninit.uniform(param.data, a=-5e-3, b=5e-3)

In [7]:
def create_batch_phase1(index, batch_size):
    step=len(data_train_phase1)//batch_size
    starting_index=index%step
    X_batch = []
    y_batch = []
    while starting_index < len(data_train_phase1):
        case, x, y, z, l = data_train_phase1[starting_index]
        case1 = case[:2]
        case2 = case[3:]
        X_batch.append(f[case1][case2][:, x-16:x+17, y-16:y+17, z])
        y_batch.append(l)
        starting_index+=step
    X_batch = torch.from_numpy(np.array(X_batch))
    y_batch = torch.from_numpy(np.array(y_batch))
    return X_batch, y_batch

def create_batch_phase2(index, batch_size):
    step=len(data_train_phase2)//batch_size
    starting_index=index%step
    X_batch = []
    y_batch = []
    while starting_index < len(data_train_phase2):
        case, x, y, z, l = data_train_phase2[starting_index]
        case1 = case[:2]
        case2 = case[3:]
        X_batch.append(f[case1][case2][:, x-16:x+17, y-16:y+17, z])
        y_batch.append(l)
        starting_index+=step
    X_batch = torch.from_numpy(np.array(X_batch))
    y_batch = torch.from_numpy(np.array(y_batch))
    return X_batch, y_batch

def create_val(batch_mask):
    X_val=[]
    y_val=[]
    for i in range(len(batch_mask)):
        case, x, y, z, l = data_val[batch_mask[i]]
        case1 = case[:2]
        case2 = case[3:]
        X_val.append(f[case1][case2][:, x-16:x+17, y-16:y+17, z])
        y_val.append(l)
    X_val = torch.from_numpy(np.array(X_val))
    y_val = torch.from_numpy(np.array(y_val))
    return X_val, y_val

In [8]:
import time
import numpy as np
num_train=len(data_train_phase1)
num_val=len(data_val)
batch_size=900
val_size=num_val
num_epoch=8.0
num_times=int(float(num_train)/batch_size*num_epoch)
print (num_times)
learning_rate = 5e-3
reg=5e-7
net = TwoPathConv()
print(net)
init_net(net)
net.cuda(1)

#create validation set
val_mask=np.random.choice(num_val, val_size)
X_val, y_val = create_val(val_mask)
X_val = Variable(X_val.cuda(1), requires_grad=False)

prev_time = time.clock()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=reg)
net.zero_grad()
for i in range(21):  # loop over the dataset multiple times
    X_batch=None
    batch_time=time.clock()
    X_batch, y_batch = create_batch_phase1(i, batch_size)
    print("batch_time: "+ str(time.clock()-batch_time))
    X_batch, y_batch = Variable(X_batch.cuda(1)), Variable(y_batch.cuda(1), requires_grad = False)
    y_pred = net.forward(X_batch)
    y_pred = y_pred.view(-1,5)
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()
    net.zero_grad()
    if i % 1000 == 0:
        print ("")
        print ('time used %.3f' % (time.clock()-prev_time))
        print ("phase1 "+str(float(i)/num_times*100)+"% completed")
        print (loss)
        y_val_pred=net.forward(X_val)
        y_val_pred=y_val_pred.view(-1,5)
        useless, predicted=torch.max(y_val_pred.data, 1)
        correct = (predicted == y_val.cuda(1)).sum()
        print('Validation accuracy:', float(correct)/val_size)
print ("phase1 successfully trained!")
torch.save(net.state_dict(), "diagnose_net.txt")
print ("phase1 successfully saved!")

76704
TwoPathConv (
  (upper_layer1): Sequential (
    (0): Conv2d(4, 64, kernel_size=(7, 7), stride=(1, 1))
    (1): ReLU ()
    (2): MaxPool2d (size=(4, 4), stride=(1, 1), dilation=(1, 1))
  )
  (upper_layer2): Sequential (
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU ()
    (2): MaxPool2d (size=(2, 2), stride=(1, 1), dilation=(1, 1))
  )
  (under_layer1): Sequential (
    (0): Conv2d(4, 160, kernel_size=(13, 13), stride=(1, 1))
    (1): ReLU ()
  )
  (final_layer): Conv2d(224, 5, kernel_size=(21, 21), stride=(1, 1))
)
batch_time: 1.845086000000009

time used 2.436
phase1 0.0% completed
Variable containing:
 1.6105
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Validation accuracy: 0.1972187104930468
batch_time: 1.8378289999999993
batch_time: 1.8515720000000044
batch_time: 1.8254750000000115
batch_time: 1.8293420000000253
batch_time: 1.848540000000014
batch_time: 1.8246469999999988
batch_time: 1.80693100000002
batch_time: 1.8235439999999983
batch_time: 1.8245

In [17]:
def adam(x, dx, config = None):
    if config is None: 
        config = {}
        config.setdefault('learning_rate',1e-4)
        config.setdefault('beta1',0.9)
        config.setdefault('beta2',0.999)
        config.setdefault('epsilon',1e-8)
        config.setdefault('m',torch.FloatTensor(x.size()).zero_().cuda(1))
        config.setdefault('v',torch.FloatTensor(x.size()).zero_().cuda(1))
        config.setdefault('t',0)
    
    next_x = None
    config['t'] += 1
    x.cuda(1)
    dx.cuda(1)
    config['m'] = config['beta1'] * config['m'] + (1 - config['beta1']) * dx
    config['v'] = config['beta2'] * config['v'] + (1 - config['beta2']) * dx**2
    mb = config['m'] / (1 - config['beta1'] ** config['t'])
    vb = config['v'] / (1 - config['beta2'] ** config['t'])
    next_x = x - config['learning_rate'] * mb / (np.sqrt(vb) + config['epsilon'])
    return next_x, config

def sgd_momentum(w, dw, config = None):

    if config is None: 
        config = {}
        config.setdefault('learning_rate', 5e-4)
        config.setdefault('momentum', 0.9)

    v = config.get('velocity', torch.FloatTensor(w.size()).zero_().cuda(1))
    next_w = None

    v = config['momentum'] * v - config['learning_rate'] * dw
    next_w = w + v
    config['velocity'] = v

    return next_w, config

In [20]:
import time
import numpy as np
num_train=len(data_train_phase2)
num_val=len(data_val)
batch_size=900
val_size=num_val
num_epoch=5.0
num_times=int(float(num_train)/batch_size*num_epoch)
print (num_times)
learning_rate = 5e-4
reg=5e-7
net.load_state_dict(torch.load("diagnose_net.txt"))
net.cuda(1)
#create validation set
val_mask=np.random.choice(num_val, val_size)
X_val, y_val = create_val(val_mask)
X_val = Variable(X_val.cuda(1), requires_grad=False)

prev_time = time.clock()
config_weight = None
config_bias = None
net.zero_grad()
for i in range(num_times):  # loop over the dataset multiple times
    X_batch = None
    X_batch, y_batch = create_batch_phase2(i, batch_size)
    X_batch, y_batch = Variable(X_batch.cuda(1)), Variable(y_batch.cuda(1), requires_grad = False)
    y_pred = net.forward(X_batch)
    y_pred = y_pred.view(-1,5)
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    dweight = net.final_layer.weight.grad.data+reg*net.final_layer.weight.data
    dbias = net.final_layer.bias.grad.data
    net.final_layer.weight.data, config_weight = sgd_momentum(net.final_layer.weight.data, dweight, config_weight)
    net.final_layer.bias.data, config_bias = sgd_momentum(net.final_layer.bias.data, dbias, config_bias)
    net.zero_grad()
    if i % 1000 == 0:
        print ("")
        print ('time used %.3f' % (time.clock()-prev_time))
        print ("phase2+ "str(float(i)/num_times*100)+"% completed")
        print (loss)
        y_val_pred=net.forward(X_val)
        y_val_pred=y_val_pred.view(-1,5)
        useless, predicted=torch.max(y_val_pred.data, 1)
        correct = (predicted == y_val.cuda(1)).sum()
        print('Validation accuracy:', float(correct)/val_size)
print ("phase2 successfully trained!")
torch.save(net.state_dict(), "diagnose_net_phase2.txt")
print ("phase2 successfully saved!")

131715

time used 5.357
0.0% completed
Variable containing:
 0.6691
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Validation accuracy: 0.8053097345132744

time used 29.911
0.0037960748585962115% completed
Variable containing:
 0.4948
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Validation accuracy: 0.8900126422250316

time used 53.996
0.007592149717192423% completed
Variable containing:
 0.3944
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Validation accuracy: 0.9190897597977244

time used 78.599
0.011388224575788634% completed
Variable containing:
 0.4175
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Validation accuracy: 0.9089759797724399

time used 102.968
0.015184299434384846% completed
Variable containing:
 0.3709
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Validation accuracy: 0.9152970922882427
phase2 successfully trained!
phase2 successfully saved!


In [6]:

# coding: utf-8

# In[1]:

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class TwoPathConv(nn.Module):
    def __init__(self):
        super(TwoPathConv, self).__init__()
        self.upper_layer1 = nn.Sequential(
            nn.Conv2d(4,64,7),
            nn.ReLU(),
            nn.MaxPool2d((4,4),stride = 1)
        )
        self.upper_layer2 = nn.Sequential(
            nn.Conv2d(64,64,3),
            nn.ReLU(),
            nn.MaxPool2d((2,2),stride = 1)
        )
        self.under_layer1 = nn.Sequential(
            nn.Conv2d(4,160,13),
            nn.ReLU()
        )
        self.final_layer = nn.Conv2d(224,5,21)
        
    def forward(self, x):
        upper_x = self.upper_layer2(self.upper_layer1(x))
        under_x = self.under_layer1(x)
        #upper_x = F.max_pool2d(F.relu(self.upper_conv1(x)), (4, 4),stride = 1)
        #upper_x = F.max_pool2d(F.relu(self.upper_conv2(upper_x)), (2, 2), stride = 1)
        #under_x = F.relu(self.under_conv1(x))
        final_x = torch.cat((under_x, upper_x), 1)
        out = self.final_layer(final_x)
        return out
        
net = TwoPathConv()
print(net)

x = Variable(torch.randn(1,4,33,33), requires_grad = True)
y_pred = net.forward(x)
print(y_pred)
#y_pred = y_pred.data.resize_(1,5)
#y_pred = Variable(y_pred,requires_grad = True)
#print(y_pred.size())
#print(y_pred.view)


# In[2]:

import h5py
f5 = h5py.File('training.h5','r')

#get the training set for phase 1
f=open("trainval-balanced.txt", "r")
content=f.readlines()
X_train_phase1=[]
y_train_phase1=[]
for line in content:
    no_n_line=line[0:len(line)-1]
    item=no_n_line.split(" ")
    case, x, y, z, l = item[0], int(item[1]), int(item[2]), int(item[3]), int(item[4])
    case1 = case[:2]
    case2 = case[3:]
    X_train_phase1.append(f5[case1][case2][:, x-16:x+17, y-16:y+17, z])
    y_train_phase1.append(l)
f.close()
print ("phase 1 data preparation process completed.")

#get the training set for phase 2 and the validation set
data_val=[]
f_in=open("trainval.txt", "r")
content=f_in.readlines()
X_train_phase2=[]
y_train_phase2=[]
X_val=[]
y_val=[]
i=0
for line in content:
    no_n_line=line[0:len(line)-1]
    item=no_n_line.split(" ")
    case, x, y, z, l = item[0], int(item[1]), int(item[2]), int(item[3]), int(item[4])
    case1 = case[:2]
    case2 = case[3:]
    if i%30000==0:
        X_val.append(f5[case1][case2][:, x-16:x+17, y-16:y+17, z])
        y_val.append(l)
    else:
        X_train_phase2.append(f5[case1][case2][:, x-16:x+17, y-16:y+17, z])
        y_train_phase2.append(l)
    i += 1
f_in.close()
print ("phase 2 data preparation process completed.")

f5.close()

# In[3]:

print (len(X_train_phase1))
print (len(y_train_phase1))
print (len(X_train_phase1))
print (len(y_train_phase2))
print (len(X_val))
print (len(y_val))

# In[4]:

print (torch.cuda.is_available())


# In[5]:

import torch.nn.init as ninit
def init_net(net):
    for param in net.parameters():
        ninit.uniform(param.data, a=-5e-3, b=5e-3)


# In[6]:

def create_batch_phase1(index, batch_size):
    start=index*batch_size
    end=start+batch_size
    X_batch= X_train_phase1[start:end]
    y_batch= y_train_phase1[start:end]
    X_batch = torch.from_numpy(np.array(X_batch))
    y_batch = torch.from_numpy(np.array(y_batch))
    return X_batch, y_batch

def create_batch_phase2(index, batch_size):
    start=index*batch_size
    end=start+batch_size
    X_batch= X_train_phase2[start:end]
    y_batch= y_train_phase2[start:end]
    X_batch = torch.from_numpy(np.array(X_batch))
    y_batch = torch.from_numpy(np.array(y_batch))
    return X_batch, y_batch

# In[7]:

import time
import numpy as np
num_train=len(data_train_phase1)
num_val=len(data_val)
batch_size=900
val_size=num_val
num_epoch=2.0
num_times=int(float(num_train)/batch_size*num_epoch)
print (num_times)
learning_rate = 5e-3
reg=5e-5
net = TwoPathConv()
print(net)
init_net(net)
net.cuda(0)

#create validation set
X_val = torch.from_numpy(np.array(X_val))
y_val = torch.from_numpy(np.array(y_val))
X_val = Variable(X_val.cuda(0), requires_grad=False)

prev_time = time.clock()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=reg)
net.zero_grad()
for i in xrange(21):  # loop over the dataset multiple times
    X_batch=None
    batch_time=time.clock()
    X_batch, y_batch = create_batch_phase1(i, batch_size)
    print("batch_time: "+ str(time.clock()-batch_time))
    X_batch, y_batch = Variable(X_batch.cuda(0)), Variable(y_batch.cuda(0), requires_grad = False)
    y_pred = net.forward(X_batch)
    y_pred = y_pred.view(-1,5)
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()
    net.zero_grad()
    if i % 5 == 0:
        print ("")
        print ('time used %.3f' % (time.clock()-prev_time))
        print (str(float(i)/num_times*100)+"% completed")
        print (loss)
        y_val_pred=net.forward(X_val)
        y_val_pred=y_val_pred.view(-1,5)
        useless, predicted=torch.max(y_val_pred.data, 1)
        correct = (predicted == y_val.cuda(0)).sum()
        print('Validation accuracy:', float(correct)/val_size)
print ("phase1 successfully trained!")
#torch.save(net.state_dict(), "premature_net.txt")
print ("phase1 successfully saved!")

# In[8]:

def adam(x, dx, config = None):
    if config is None: 
        config = {}
        config.setdefault('learning_rate',1e-4)
        config.setdefault('beta1',0.9)
        config.setdefault('beta2',0.999)
        config.setdefault('epsilon',1e-8)
        config.setdefault('m',torch.FloatTensor(x.size()).zero_().cuda(0))
        config.setdefault('v',torch.FloatTensor(x.size()).zero_().cuda(0))
        config.setdefault('t',0)
    
    next_x = None
    config['t'] += 1
    x.cuda(0)
    dx.cuda(0)
    config['m'] = config['beta1'] * config['m'] + (1 - config['beta1']) * dx
    config['v'] = config['beta2'] * config['v'] + (1 - config['beta2']) * dx**2
    mb = config['m'] / (1 - config['beta1'] ** config['t'])
    vb = config['v'] / (1 - config['beta2'] ** config['t'])
    next_x = x - config['learning_rate'] * mb / (np.sqrt(vb) + config['epsilon'])
    return next_x, config

def sgd_momentum(w, dw, config = None):

    if config is None: 
        config = {}
        config.setdefault('learning_rate', 5e-4)
        config.setdefault('momentum', 0.9)

    v = config.get('velocity', torch.FloatTensor(w.size()).zero_().cuda(0))
    next_w = None

    v = config['momentum'] * v - config['learning_rate'] * dw
    next_w = w + v
    config['velocity'] = v

    return next_w, config


# In[11]:

import time
import numpy as np
num_train=len(data_train_phase2)
num_val=len(data_val)
batch_size=900
val_size=num_val
num_epoch=2.0
num_times=int(float(num_train)/batch_size*num_epoch)
print (num_times)
learning_rate = 5e-4
reg=5e-5
net.load_state_dict(torch.load("premature_net.txt"))
net.cuda(0)

prev_time = time.clock()
config_weight = None
config_bias = None
net.zero_grad()
for i in xrange(21):  # loop over the dataset multiple times
    X_batch = None
    batch_time=time.clock()
    X_batch, y_batch = create_batch_phase2(i, batch_size)
    print("batch_time: "+ str(time.clock()-batch_time))
    X_batch, y_batch = Variable(X_batch.cuda(0)), Variable(y_batch.cuda(0), requires_grad = False)
    y_pred = net.forward(X_batch)
    y_pred = y_pred.view(-1,5)
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    dweight = net.final_layer.weight.grad.data+reg*net.final_layer.weight.data
    dbias = net.final_layer.bias.grad.data
    net.final_layer.weight.data, config_weight = sgd_momentum(net.final_layer.weight.data, dweight, config_weight)
    net.final_layer.bias.data, config_bias = sgd_momentum(net.final_layer.bias.data, dbias, config_bias)
    net.zero_grad()
    if i % 5 == 0:
        print ("")
        print ('time used %.3f' % (time.clock()-prev_time))
        print (str(float(i)/num_times*100)+"% completed")
        print (loss)
        y_val_pred=net.forward(X_val)
        y_val_pred=y_val_pred.view(-1,5)
        useless, predicted=torch.max(y_val_pred.data, 1)
        correct = (predicted == y_val.cuda(0)).sum()
        print('Validation accuracy:', float(correct)/val_size)
print ("phase2 successfully trained!")
torch.save(net.state_dict(), "premature_net_phase2_without_biasreg.txt")
print ("phase2 successfully saved!")

TwoPathConv (
  (upper_layer1): Sequential (
    (0): Conv2d(4, 64, kernel_size=(7, 7), stride=(1, 1))
    (1): ReLU ()
    (2): MaxPool2d (size=(4, 4), stride=(1, 1), dilation=(1, 1))
  )
  (upper_layer2): Sequential (
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU ()
    (2): MaxPool2d (size=(2, 2), stride=(1, 1), dilation=(1, 1))
  )
  (under_layer1): Sequential (
    (0): Conv2d(4, 160, kernel_size=(13, 13), stride=(1, 1))
    (1): ReLU ()
  )
  (final_layer): Conv2d(224, 5, kernel_size=(21, 21), stride=(1, 1))
)
Variable containing:
(0 ,0 ,.,.) = 
  0.0960

(0 ,1 ,.,.) = 
  0.1640

(0 ,2 ,.,.) = 
 -0.1697

(0 ,3 ,.,.) = 
 -0.1479

(0 ,4 ,.,.) = 
  0.1214
[torch.FloatTensor of size 1x5x1x1]



KeyboardInterrupt: 